# ETL Master Flight Table Using PySpark
The master flight table combines attributes from external tables to form a single flat table to use as a starting point for training our ML models. External attribute tables such as airport demand and weather can be pre-partitioned by airport name. External tables are typically joined to the master flight table twice: once for arrivals and a second time for departures. Data alignment is required to merge these tables so multiple shuffle operations are required for each of the joins. Pre-partitioning the data ensures that the dominant join pattern can take advantage of less expensive, in-block operations to avoid unnecessary data movement. This becomes critical for large datasets in a distributed cluster environment.  

Hand tuning data flows requires a deep understanding of internal data structures. Operations that worked for small data samples may not be scalable to the full dataset. This provides some motivation to try to improve performance using advanced libraries such as spark, dask, or NVTabular to enable us to reuse simple code and scale it to the full dataset without having to resort to writing additional custom code or patches for resolving performance issues at scale. With the RAPIDS plugin for spark, the same code used on CPU deployments can be reused on the GPU to accelerate the ETL pipeline. 

In [1]:
import os
import shutil
from time import time
import pandas as pd
import numpy as np
from functools import reduce


# Load external tables for merging:
staging_dir = './data/staging_tbl/'
etl_output_dir = './data/encoded/NAS/' # Directory will be wiped!
os.makedirs(etl_output_dir, exist_ok = True)

try:
    # RECURSIVELY DELETE DIRECTORY and then add it
    shutil.rmtree(etl_output_dir)
except:
    pass

os.mkdir(etl_output_dir)

import pyspark
from pyspark.sql.functions import col

spark_cluster = 'local'

if spark_cluster == 'local':
    num_exec = 32
    spark = (pyspark.sql.SparkSession.builder
             .master('local['+str(num_exec)+']')
             .config('spark.executor.memory', '50g')
             .config('spark.driver.memory', '150g')
             .config('spark.sql.adaptive.enable', 'true')
             .config('spark.sql.shuffle.partitions', 10*num_exec)
             .config('spark.sql.execution.arrow.pyspark.enabled', 'true')
             .config('spark.local.dir', './tmp/spark_tmp/') # Optional. Main drive was filling up.
            ).getOrCreate()
else:
    spark_master = 'spark://boxx-wlo2:7077' # Cluster
    spark = (pyspark.sql.SparkSession.builder
             .master(spark_master)
             .config('spark.sql.shuffle.partitions', 32)
             .config('spark.sql.execution.arrow.pyspark.enabled', 'true')
             .config('spark.local.dir', './tmp/spark_tmp/')  # Optional. Main drive was filling up.
            ).getOrCreate()


spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/28 18:30:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/28 18:30:08 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


# Spark Merge

In [2]:
%%time

def spark_shape(df):
    return('(' + f'{df.count():,}' + ' ' +str(len(df.columns)) +')' )

drop_meta_cols = ['__null_dask_index__', '__index_level_0__']

arpt_demand = spark.read.parquet(staging_dir+'arpt_demand').drop(*drop_meta_cols, 'YEAR').cache()
print('Airport demand shape:', spark_shape(arpt_demand))

arpt_weather = spark.read.parquet(staging_dir+'arpt_weather').drop(*drop_meta_cols, 'YEAR').cache()
print('Airport weather shape:', spark_shape(arpt_weather))

nas_flights = spark.read.parquet(staging_dir+'nas_flights').drop(*drop_meta_cols).cache()
print('Flight table shape:', spark_shape(nas_flights))

22/01/28 18:30:12 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Airport demand shape: (1,989,473 34)
Airport weather shape: (568,916 16)


Flight table shape: (1,555,982 61)
CPU times: user 26.7 ms, sys: 7.79 ms, total: 34.5 ms
Wall time: 11.8 s


In [3]:
%%time

def add_col_prefix(df, update_col_names, prefix, ignore_cols):
    cols_with_prefix = [cc for cc in update_col_names if cc not in ignore_cols]
    col_rename_map = list(zip(cols_with_prefix, [prefix+cc for cc in cols_with_prefix]))
    
    for cc in col_rename_map:
        df = df.withColumnRenamed(*cc)
    return(df)

# Create master flight table by combining attributes from ORIGIN/DEST airports.
# Merge and rename new columns afterward involves 3 table scan instead of 5. External tables can be cached for ease of reuse.

#####################################
## Merge Airport Demand Attributes ##
#####################################
# ORIGIN airport demand:
nas_flights_mg = (nas_flights.join(arpt_demand, 
                                  (nas_flights['ORIGIN']==arpt_demand['ARPT_NAME']) & 
                                  (nas_flights['DEP_TIME_DT_LOCAL_QTHR']==arpt_demand['DT_LOCAL_QTHR']), 'left')
                  .drop('ARPT_NAME', 'DT_LOCAL_QTHR')
                 )
nas_flights_mg = add_col_prefix(nas_flights_mg, arpt_demand.columns, 'ORIGIN_', ['ARPT_NAME', 'DT_LOCAL_QTHR'])

# DEST airport demand:
nas_flights_mg = (nas_flights_mg.join(arpt_demand, 
                                  (nas_flights_mg['DEST']==arpt_demand['ARPT_NAME']) & 
                                  (nas_flights_mg['ARR_TIME_DT_LOCAL_QTHR']==arpt_demand['DT_LOCAL_QTHR']), 'left')
                  .drop('ARPT_NAME', 'DT_LOCAL_QTHR')
                 )
nas_flights_mg = add_col_prefix(nas_flights_mg, arpt_demand.columns, 'DEST_', ['ARPT_NAME', 'DT_LOCAL_QTHR'])


######################################
## Merge Airport Weather Attributes ##
######################################
# ORIGIN airport weather:
nas_flights_mg = (nas_flights_mg.join(arpt_weather, 
                                      (nas_flights_mg['ORIGIN']==arpt_weather['ARPT_NAME']) &
                                      (nas_flights_mg['DEP_TIME_DT_LOCAL_HR']==arpt_weather['DT_LOCAL_HR']), 'left')
                  .drop('ARPT_NAME', 'DT_LOCAL_HR')
                 )
nas_flights_mg = add_col_prefix(nas_flights_mg, arpt_weather.columns, 'ORIGIN_', ['ARPT_NAME', 'DT_LOCAL_HR'])

# DEST airport weather:
nas_flights_mg = (nas_flights_mg.join(arpt_weather, 
                                      (nas_flights_mg['DEST']==arpt_weather['ARPT_NAME']) &
                                      (nas_flights_mg['ARR_TIME_DT_LOCAL_HR']==arpt_weather['DT_LOCAL_HR']), 'left')
                  .drop('ARPT_NAME', 'DT_LOCAL_HR')
                 )
nas_flights_mg = add_col_prefix(nas_flights_mg, arpt_weather.columns, 'DEST_', ['ARPT_NAME', 'DT_LOCAL_HR'])

###################
## Clean Columns ##
###################
arpt_demand_attr_cols = [cc for cc in arpt_demand.columns if cc not in ['YEAR', 'ARPT_NAME', 'DT_LOCAL_QTHR']]
arpt_demand_attr_cols = [prefix + dcols for prefix in ['ORIGIN_', 'DEST_'] for dcols in arpt_demand_attr_cols]

# Fill missing data due to merging:
nas_flights_mg = nas_flights_mg.fillna(0, subset=arpt_demand_attr_cols)

# Due to sporadic weather data, missing fields can only be dropped instead of imputed.

# Remove certain cols with ORIGIN_/DEST_ prefix:
od_remove_cols = ['DT_LOCAL_QTHR', 'DT_LOCAL_HR']
od_remove_cols = [pfix+cc for pfix in ['ORIGIN_', 'DEST_'] for cc in od_remove_cols]

# Remove certain cols with ARR/DEP_ prefix:
ad_remove_cols = ['TIME_DT_LOCAL', 'TIME_DT_LOCAL_DAY', 'TIME_DT_LOCAL_HR', 'TIME_DT_LOCAL_QTHR']
ad_remove_cols = [pfix+cc for pfix in ['ARR_', 'DEP_'] for cc in ad_remove_cols]

# Drop unecessary columns to prevent leakage and duplicate data:
nas_flights_mg = nas_flights_mg.drop(*(ad_remove_cols + od_remove_cols))

# Using coalesce(1) results in single task and reduced parallelism. repartition() involves shuffling, but is parallel.
# Not applying coalesce/repartion results in large number of small files (equal to the number of spark.sql.shuffle.partitions).
nas_flights_mg.coalesce(4).write.mode('overwrite').partitionBy('YYYYMM').parquet(etl_output_dir)

# TODO: optimize merging.
# Chain merge of 17-year data took around 17 minutes on 2x rtx8000 vs 35 minutes for 32-core CPU run.
# Chain merge of 32-year data took around xx minutes on 2x rtx8000 vs 96 minutes for 32-core CPU run.

CPU times: user 48.2 ms, sys: 0 ns, total: 48.2 ms
Wall time: 19.2 s
